In [174]:
import sympy
from sympy import *
from sympy import init_session
init_session()

IPython console for SymPy 0.7.6.1 (Python 3.5.1-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at http://www.sympy.org


In [175]:
def func_gradient(func, theta_sym, values):
    grad = np.array([diff(L(*theta_sym), i) for i in theta_sym])
    substitutes = list(zip(theta_sym, values))

    for i, element in enumerate(grad):
        for j in substitutes:
            grad[i] = grad[i].subs(*j)
    return grad  

In [185]:
def L(x,y):
    return x**2*y**2 + y**2

theta_sym = [x,y]
values = [2,4]

out, _ = func_gradient(L, theta_sym, values)
out

In [120]:
# from Algorithm 1: HMC, page1353
def Leapfrog(theta, r, eps, theta_sym):
    r_tilde = r + (eps/2) * func_gradient(L, theta_sym, values = theta)
    theta_tilde = theta + eps * r_tilde
    r_tide = r_tilde + (eps/2) * func_gradient(L, theta_sym, values = theta_tilde)
    return theta_tilde, r_tilde

(array([5.00000000000000, 9.00000000000000], dtype=object),
 array([3.00000000000000, 5.00000000000000], dtype=object))

In [187]:
Leapfrog(np.array([1,2]), 1, 1, [x,y])

(array([6.00000000000000, 7.00000000000000], dtype=object),
 array([5.00000000000000, 5.00000000000000], dtype=object))

In [184]:
def BuildTree(theta, r, u, v, j, eps):
    if j == 0:
        # base case, take one leapfrog step in the direction v
        theta_prime, r_prime = Leapfrog(theta, r, v*eps, theta_sym)
        
        if u <= np.exp(L(*theta_prime) - 0.5 * np.dot(r_prime, r_prime)):
            C_prime = np.append(theta_prime, r_prime)
        else:
            C_prime = np.empty(shape=(0, 0))
        
        s_prime = int(L(*theta_prime) - 0.5 * np.dot(r_prime,r_prime) > np.log(u) - 1000)
        
        return theta_prime, r_prime, theta_prime, r_prime, C_prime, s_prime
    else:
        # recursion, build left and right subtrees
        theta_minus, r_minus, theta_plus, r_plus, C_prime, s_prime = BuildTree(theta, r, u, v, j-1, eps)
        
        if v == -1:
            theta_minus, r_minus, _, _, C_doub_prime, s_doub_prime = BuildTree(theta_minus, r_minus, u, v, j-1, eps)
        else:
            _,_, theta_plus, r_plus, C_doub_prime, s_doub_prime = BuildTree(theta_plus, r_plus, u, v, j-1, eps)
            
        ind_1 = int(np.dot(theta_plus-theta_minus, r_minus) >= 0)
        ind_2 = int(np.dot(theta_plus-theta_minus, r_plus) >= 0)
        s_prime = s_prime * s_doub_prime * ind_1 * ind_2
        C_prime = np.append(C_prime, C_doub_prime)
        
        return theta_minus, r_minus, theta_plus, r_plus, C_prime, s_prime

In [ ]:
#BuildTree(np.array([1,2]), np.array([3,4]))

In [186]:
one = np.array([1,2])
two = np.array([3,4])
values

In [141]:
np.append(one,two)

array([1, 2, 3, 4])

In [165]:
esto = 2 > 3
1 * esto

In [166]:
int(esto)

In [179]:
2 * 3

SyntaxError: invalid syntax (<ipython-input-182-139d1a51aa9c>, line 1)